In [1]:
# Import key packages
import numpy as np
import iodata
from iodata import IOData
import gbasis
from gbasis.wrappers import from_iodata
from iodata import load_one, dump_one
from pyscf import gto, scf
from pyscf.scf.chkfile import dump_scf
import pyci

In [2]:
def RanGo(N):
    h2 = np.zeros((2*N, 2*N, 2*N, 2*N))
    
    # build hamiltonian, i and k are uparrow, <ij|kl>
    # a b a b
    for i in range(2*N):
        for j in range(N,2*N):
            for k in range(2*N):
                for l in range(N,2*N):
                    delta_ij = 1 if i == j+N or i+N == j else 0
                    delta_kl = 1 if k == l+N or k+N == l else 0
                    delta_il = 1 if i == k else 0
                    h2[i, j, k, l] = delta_ij * delta_kl #* (1 - delta_il)
    # b a b a 
    for i in range(N,2*N):
        for j in range(N):
            for k in range(N,2*N):
                for l in range(N):
                    delta_ij = 1 if i == j+N or i+N == j else 0
                    delta_kl = 1 if k == l+N or k+N == l else 0
                    delta_il = 1 if i == k else 0
                    h2[i, j, k, l] = delta_ij * delta_kl #* (1 - delta_il)

    return h2

In [3]:
def Vee(eri,dm2):
    return .5*np.einsum('pqrs,pqrs', eri[0], dm2[0])+.5*np.einsum('pqrs,pqrs', eri[2], dm2[2])+np.einsum('pqrs,pqrs', eri[1], dm2[1])

In [4]:
from moha.hamiltonians import HamHub, HamRG

g= -1.0

M = 6
nelec = M//2
adjacency = np.ones((M,M))
Richardson = HamRG(np.array(range(M))*0, g, adjacency)

h0_ric = Richardson.generate_zero_body_integral()
h1_ric = Richardson.generate_one_body_integral(basis='spinorbital basis', dense=True)
h0_ric = h0_ric*0
h1_ric = h1_ric*0

# h2_ric = Richardson.generate_two_body_integral(sym=8,basis='spinorbital basis',
#                                                 dense=True)
h2_ric = RanGo(M)*g

print(h2_ric.shape)

ham_ric_int = pyci.secondquant_op(h0_ric, h1_ric, h2_ric)


(12, 12, 12, 12)


In [5]:
# Check different approaches using CISD.
# Note that the Hamiltonian is (supposed to be) the same in all cases.
occs = (nelec//2,nelec//2)
excitations = (0, 1, 2, 3, 4, 5, 6, 7, 8 )     # excitations to include (0 = reference, 1 = single, 2 = double)

# From PySCF (integrals)
wfn_ric_int = pyci.fullci_wfn(ham_ric_int.nbasis, *occs)
pyci.add_excitations(wfn_ric_int, *excitations, ref=None)

op = pyci.sparse_op(ham_ric_int, wfn_ric_int)
e_vals_ric_int, e_vecs_ric_int = op.solve(n=1, tol=1.0e-9)
print("E Ric (integrals): ", e_vals_ric_int[0])

E Ric (integrals):  -18.000000000000007


In [6]:
d1, d2 = pyci.compute_rdms(wfn_ric_int, e_vecs_ric_int[0])
print(np.einsum('pqpq',d2[0]+2*d2[1]+d2[2]))

print(e_vals_ric_int[0]-np.trace(h1_ric@d1[0])-np.trace(h1_ric@d1[0])-h0_ric)
print(2*Vee((h2_ric*0,h2_ric,h2_ric*0),d2))

26.999999999999964
-18.000000000000007
-9.333333333333336
